# Fisher test

In [1]:
import os, sys, time
import numpy as np
import scipy as sp
import healpy as hp
from fgbuster.observation_helpers import *

import micmac

# Matrix operations

In [2]:
OPTIMIZE = 'optimal'
def _inv(m):
    result = np.array(map(np.linalg.inv, m.reshape((-1,)+m.shape[-2:])))
    return result.reshape(m.shape)


def _mv(m, v):
    return np.einsum('...ij,...j->...i', m, v, optimize=OPTIMIZE)

#Added by Clement Leloup                                                                                                                                                                                    
def _utv(u, v):
    return np.einsum('...i,...i', u, v)
def _uvt(u, v):
    return np.einsum('...i,...j->...ij', u, v)
def _utm(u, m):
    return np.einsum('...i,...ij->...j', u, m, optimize=OPTIMIZE)

def _utmv(u, m, v):
    return np.einsum('...i,...ij,...j', u, m, v, optimize=OPTIMIZE)

def _mtv(m, v):
    return np.einsum('...ji,...j->...i', m, v, optimize=OPTIMIZE)

def _mm(m, n):
    return np.einsum('...ij,...jk->...ik', m, n, optimize=OPTIMIZE)


def _mtm(m, n):
    return np.einsum('...ji,...jk->...ik', m, n, optimize=OPTIMIZE)


def _mmv(m, w, v):
    return np.einsum('...ij,...jk,...k->...i', m, w, v, optimize=OPTIMIZE)


def _mtmv(m, w, v):
    return np.einsum('...ji,...jk,...k->...i', m, w, v, optimize=OPTIMIZE)


def _mmm(m, w, n):
    return np.einsum('...ij,...jk,...kh->...ih', m, w, n, optimize=OPTIMIZE)


def _mtmm(m, w, n):
    return np.einsum('...ji,...jk,...kh->...ih', m, w, n, optimize=OPTIMIZE)

# Getting right format

In [3]:
def _format_prior(c_ells, lmax):

    ell = hp.Alm.getlm(lmax)[0]
    ell = np.stack((ell, ell), axis=-1).reshape(-1)

    c_ellm = np.array([c_ells[l,:] for l in ell])#[:,np.newaxis,:]
    c_ellm[np.arange(1, 2*(lmax+1), 2), ...] = 0 # to avoid overcounting for m=0                                                                                                                                
    # c_ellm = np.swapaxes(c_ellm, 0, -1)
    return c_ellm

def _format_alms(alms_c, lmin=0):
    
    alms = alms_c * 1. # Wang: extra line added                                                                                                                                                             
    lmax = hp.Alm.getlmax(alms.shape[-1])
    alms = np.asarray(alms, order='C')
    alms = alms.view(np.float64)
    em = hp.Alm.getlm(lmax)[1]
    em = np.stack((em, em), axis=-1).reshape(-1)
    #alms *= (-1)**em   # Should be there in principle but when no oupling between m's, alms only appear squared in the likelihood                                                                          
    mask_em = [m != 0 for m in em]
    alms[..., mask_em] *= np.sqrt(2)
    alms[..., np.arange(1, 2*(lmax+1), 2)] = hp.UNSEEN  # Mask imaginary m = 0                                                                                                                              
    # mask_alms = _intersect_mask(alms)
    # alms[..., mask_alms] = 0  # Thus no contribution to the spectral likelihood                                                                                                                             
    alms = np.swapaxes(alms, 0, -1)

    if lmin != 0:
        ell = hp.Alm.getlm(lmax)[0]
        ell = np.stack((ell, ell), axis=-1).reshape(-1)
        mask_lmin = [l < lmin for l in ell]
        alms[mask_lmin, ...] = 0

    return alms

def _get_alms(data_map, beams=None, lmax=None, weights=None, iter=3):
    alms = []
    for f, fdata in enumerate(data_map):
        if weights is None:
            alms.append(hp.map2alm(fdata, lmax=lmax, iter=iter))
        else:
            alms.append(hp.map2alm(hp.ma(fdata)*weights, lmax=lmax, iter=iter))
            # logging.info(f"{f+1} of {len(data)} complete")
        print(f"{f+1} of {len(data_map)} complete",flush=True)
    alms = np.array(alms)

    if beams is not None:
        # logging.info('Correcting alms for the beams')
        print('Correcting alms for the beams',flush=True)
        for fwhm, alm in zip(beams, alms):
            bl = hp.gauss_beam(np.radians(fwhm/60.0), lmax, pol=(alm.ndim==2))

            if alm.ndim == 1:
                alm = [alm]
                bl = [bl]

            for i_alm, i_bl in zip(alm, bl.T):
                hp.almxfl(i_alm, 1.0/i_bl, inplace=True)

    return alms


# Compute Fisher

In [4]:
def nonparam_biascorr_fisher_bruteforce(A, s, Sc, Sc_approx, A_dB, Sc_dr, invN=None):

    Fisher = np.zeros((len(A_dB)+1, len(A_dB)+1))

    try:
        cov = np.linalg.inv(_mtmm(A, invN, A))
    except np.linalg.LinAlgError:
        cov = np.zeros_like(_mtmm(A, invN, A))
        good_idx = np.where(np.all(np.diagonal(_mtmm(A, invN, A), axis1=-1, axis2=-2), axis=-1))
        cov[good_idx] = np.linalg.inv(_mtmm(A, invN[good_idx], A))

    Nc = cov[..., 0, 0]
    P = invN - _mmm(_mm(invN, A), cov, _mtm(A, invN))

    #Define CMB average of s*T(s)                                                                                                                                                                           
    sst_avg = np.zeros((Sc.shape[0], s.shape[1], s.shape[-1], s.shape[-1]))
    sst_avg[..., 0, 0] = Sc
    sst_avg[..., 1:, 1:] = _uvt(s[..., 1:], s[..., 1:])

    for i in np.arange(len(A_dB)):
        A_dB_i = A_dB[i]
        print('A_dB = ', A_dB_i)
        for j in np.arange(len(A_dB)):
            A_dB_j = A_dB[j]

            Nc_dB = -2.0*_mmm(cov, _mtmm(A_dB_i, invN, A), cov)[..., 0, 0]
            Nc_dB_prime = -2.0*_mmm(cov, _mtmm(A_dB_j, invN, A), cov)[..., 0, 0]

            with np.errstate(divide='ignore'):
                m1_dBdB = 2.0*np.trace(_mtmm(A_dB_i, P, _mm(A_dB_j, sst_avg)), axis1=-1, axis2=-2) - Nc_dB*Nc_dB_prime/((Sc+Nc)**2) + Nc_dB*Nc_dB_prime/((Sc_approx+Nc)**2)
                m2_dBdB = 2.0*(Sc-Sc_approx)/((Sc_approx+Nc)*(Sc+Nc))*_mmm(cov, _mmm(_mtmm(A, invN, A_dB_i), cov, _mtmm(A, invN, A_dB_j)) 
                                                                           + _mmm(_mtmm(A, invN, A_dB_j), cov, _mtmm(A, invN, A_dB_i)) 
                                                                           + _mmm(_mtmm(A, invN, A_dB_i), cov, _mtmm(A_dB_j, invN, A)) 
                                                                           - _mtmm(A_dB_i, P, A_dB_j), cov)[..., 0, 0]
                m3_dBdB = 2.0/(Sc+Nc)*_mmm(cov, _mmm(_mtmm(A, invN, A_dB_i), sst_avg, _mtmm(A_dB_j, invN, A)), cov)[..., 0, 0]
            m1_dBdB[~np.isfinite(m1_dBdB)] = 0.
            m2_dBdB[~np.isfinite(m2_dBdB)] = 0.
            m3_dBdB[~np.isfinite(m3_dBdB)] = 0.
            Fisher[i, j] = np.sum(m1_dBdB+m2_dBdB+m3_dBdB)

    m_drdr = np.zeros(Sc.shape)
    with np.errstate(divide='ignore'):
        m_drdr = (Sc_dr/(Sc+Nc))**2
    m_drdr[~np.isfinite(m_drdr)] = 0.
    Fisher[-1, -1] = np.sum(m_drdr)
    print(Fisher)

    w, v = np.linalg.eig(Fisher)
    print("eigenvalues : ", w)
    cond = np.linalg.cond(Fisher)
    print("condition number : ", cond)

    return Fisher


In [5]:


# invNlm = np.array([inv_Nl[l,:,:] for l in ell])[:,np.newaxis,:,:]
# invNlm[np.arange(1, 2*(lmax+1), 2), ...] = 0 # to avoid overcounting for m=0                                                                                                                                
# invNlm = invNlm[mask_lmin, ...]




In [6]:
working_directory_path = os.path.abspath('') + '/'
# directory_save_file = working_directory_path + 'save_directory/'
directory_toml_file = working_directory_path #+ 'toml_params/'
path_toml_file = directory_toml_file + 'full_chain_v1a.toml'
path_toml_file = directory_toml_file + 'LiteBIRD_full_chain_v1a.toml'


In [7]:
MICMAC_sampler_obj = micmac.create_MICMAC_sampler_from_toml_file(path_toml_file)

In [26]:
# General parameters
cmb_model = 'c1'
cmb_model = ''
fgs_model = 'd0s0'
model = cmb_model+fgs_model
noise = True
# noise = False
noise_seed = 42
# instr_name = dictionary_parameters['instrument_name'] #'SO_SAT'
instr_name = 'LiteBIRD'

# get instrument from public database
instrument = get_instrument(instr_name)

# get input freq maps
np.random.seed(noise_seed)
# freq_maps = get_observation(instrument, model, nside=NSIDE, noise=noise)[:, 1:, :]   # keep only Q and U
freq_maps_fgs = get_observation(instrument, fgs_model, nside=MICMAC_sampler_obj.nside, noise=noise)[:, 1:, :]   # keep only Q and U
print("Shape for input frequency maps :", freq_maps_fgs.shape)

fgbuster_input_freq_maps = get_observation(instrument, model, nside=MICMAC_sampler_obj.nside, noise=noise)[:, 1:, :]   # keep only Q and U

Shape for input frequency maps : (15, 2, 49152)


In [27]:
init_mixing_matrix_obj = micmac.InitMixingMatrix(MICMAC_sampler_obj.frequency_array, MICMAC_sampler_obj.number_components, pos_special_freqs=MICMAC_sampler_obj.pos_special_freqs)

exact_params_mixing_matrix = init_mixing_matrix_obj.init_params()

mixing_matrix_obj = micmac.MixingMatrix(MICMAC_sampler_obj.frequency_array, MICMAC_sampler_obj.number_components, exact_params_mixing_matrix, pos_special_freqs=MICMAC_sampler_obj.pos_special_freqs)
invBtinvNB = micmac.get_inv_BtinvNB(MICMAC_sampler_obj.freq_inverse_noise, mixing_matrix_obj.get_B())

In [28]:
def get_true_Cl_noise(depth_p, lmax):
    bl = np.ones((len(depth_p), lmax+1))

    nl = (bl / np.radians(depth_p/60.)[:, np.newaxis])**2
    return np.einsum('fk,kl->fkl', np.eye(len(depth_p)), nl)
    # return nl#.swapaxes(-2, -1)

In [29]:
# inv_Nl = micmac.get_Cl_noise(np.array(instrument['depth_p']), mixing_matrix_obj.get_B(), lmax=MICMAC_sampler_obj.lmax)
inv_Nl = get_true_Cl_noise(np.array(instrument['depth_p']), lmax=MICMAC_sampler_obj.lmax)

In [30]:
inv_Nl.shape

(15, 15, 129)

In [31]:
input_freq_maps, input_cmb_maps, theoretical_red_cov_r0_total, theoretical_red_cov_r1_tensor = MICMAC_sampler_obj.generate_input_freq_maps_from_fgs(freq_maps_fgs, return_only_freq_maps=False)

Calculating spectra from CAMB !
Calculating spectra from CAMB !


In [32]:
# data_map = input_freq_maps
data_map = fgbuster_input_freq_maps

In [33]:

theoretical_r0_total = micmac.get_c_ells_from_red_covariance_matrix(theoretical_red_cov_r0_total)#[partial_indices_polar,:]
theoretical_r1_tensor = micmac.get_c_ells_from_red_covariance_matrix(theoretical_red_cov_r1_tensor)#[partial_indices_polar,:]

indices_polar = np.array([1,2,4])
partial_indices_polar = indices_polar[:MICMAC_sampler_obj.nstokes]

c_ell_approx = np.zeros((6, MICMAC_sampler_obj.lmax+1))
c_ell_approx[indices_polar,MICMAC_sampler_obj.lmin:] = theoretical_r0_total

Cl_lens = np.zeros_like(c_ell_approx)
Cl_lens[indices_polar,MICMAC_sampler_obj.lmin:] = theoretical_r0_total

Cl_prim = np.zeros_like(c_ell_approx)
Cl_prim[indices_polar,MICMAC_sampler_obj.lmin:] = theoretical_r1_tensor

In [34]:
exact_params_mixing_matrix, exact_params_mixing_matrix.ravel(order='F')

(array([[0.52387813, 0.00182426],
        [0.3117258 , 0.00324908],
        [0.21971687, 0.00433627],
        [0.15096497, 0.0057459 ],
        [0.10631274, 0.0074421 ],
        [0.07881344, 0.00935487],
        [0.05159725, 0.0133358 ],
        [0.03585547, 0.01906532],
        [0.02552178, 0.02888587],
        [0.0193825 , 0.04508262],
        [0.01493871, 0.08217483],
        [0.01211497, 0.16061503],
        [0.00871575, 0.37616284]]),
 array([0.52387813, 0.3117258 , 0.21971687, 0.15096497, 0.10631274,
        0.07881344, 0.05159725, 0.03585547, 0.02552178, 0.0193825 ,
        0.01493871, 0.01211497, 0.00871575, 0.00182426, 0.00324908,
        0.00433627, 0.0057459 , 0.0074421 , 0.00935487, 0.0133358 ,
        0.01906532, 0.02888587, 0.04508262, 0.08217483, 0.16061503,
        0.37616284]))

In [35]:
# ell = hp.Alm.getlm(MICMAC_sampler_obj.lmax)[0]
# np.array([Cl_prim[l,:,:] for l in ell])[:,np.newaxis,:,:]

In [36]:
# c_ell_approx = np.array([c_ell_approx for freq in range(MICMAC_sampler_obj.number_frequencies)])
# Cl_lens = np.array([Cl_lens for freq in range(MICMAC_sampler_obj.number_frequencies)])
# Cl_prim = np.array([Cl_prim for freq in range(MICMAC_sampler_obj.number_frequencies)])
# inv_Nl = np.array([inv_Nl for freq in range(MICMAC_sampler_obj.number_frequencies)])
data_map_extended = np.insert(data_map, [0], np.zeros(MICMAC_sampler_obj.npix), axis=1)


In [37]:

def full_fisher(init_mixing_matrix_obj, exact_params_mixing_matrix, Cl_lens, Cl_prim, data_map, lmax, inv_Nl, lmin=2, r_start=0.001, mode='EB', Sc_approx=None):
    
    # Format objects                                                                                                                                                                                            
    ell = hp.Alm.getlm(lmax)[0]
    ell = np.stack((ell, ell), axis=-1).reshape(-1)
    mask_lmin = [l >= lmin for l in ell]
    
    alms = _get_alms(data_map, lmax=lmax)[:, 1:, :]
    alms = _format_alms(alms, lmin=0)
    alms = alms[mask_lmin, ...]
    # print("Test alms", alms.shape, flush=True)

    invNlm = np.array([inv_Nl.T[l,:,:] for l in ell])[:,np.newaxis,:,:]
    invNlm[np.arange(1, 2*(lmax+1), 2), ...] = 0 # to avoid overcounting for m=0                                                                                                                                
    invNlm = invNlm[mask_lmin, ...]
    # nblind : number of non-param components in the mixing matrix                                                                                                                                          

    # print("Test Nlm", invNlm.shape, flush=True)

    assert mode in ['B', 'E', 'EB']

    # if nblind is None:
	# nblind = len(components)-1
    # nfreq = np.array([0, -1]) #useless

    print('Computing mixing matrix')
    # A_ev, B_dB_matrix, comp_of_param, x0, params = init_nonparam_mixmat(components, instrument, nblind, nfreq=nfreq) #useless, to be replaced by MICMAC mixing matrix
    B_matrix = init_mixing_matrix_obj.get_B()
    B_dB_matrix = init_mixing_matrix_obj.get_B_db()
    # if not len(x0):
    #     print("No spectral parameter to maximize !")

    # x0 = np.append(x0, r_start) # true values of the parameters
    x0 = np.append(np.ravel(exact_params_mixing_matrix,order='F'), r_start) # true values of the parameters                                                                                                                                             


    # Format C and C_approx                                                                                                                                                                                 
    ell_in = np.arange(2, lmax+1)
    mask_lmin = [l >= lmin for l in ell]

    if mode == 'B': # B modes only                                                                                                                                                                          
        Cl_prim = _format_prior(Cl_prim[2:3,:lmax+1].T, lmax)
        Cl_lens = _format_prior(Cl_lens[2:3,:lmax+1].T, lmax)
        if Sc_approx is not None:
            Sc_approx = _format_prior(Sc_approx[2:3,:lmax+1].T, lmax)
    elif mode == 'E': # E modes only                                                                                                                                                                        
        Cl_prim = _format_prior(Cl_prim[1:2,:lmax+1].T, lmax)
        Cl_lens = _format_prior(Cl_lens[1:2,:lmax+1].T, lmax)
        if Sc_approx is not None:
            Sc_approx = _format_prior(Sc_approx[1:2,:lmax+1].T, lmax)
    else: # E and B modes                                                                                                                                                                                   
        Cl_prim = _format_prior(Cl_prim[1:3,:lmax+1].T, lmax)
        Cl_lens = _format_prior(Cl_lens[1:3,:lmax+1].T, lmax)
        if Sc_approx is not None:
            Sc_approx = _format_prior(Sc_approx[1:3,:lmax+1].T, lmax)


    # print("Test", Cl_prim.shape, flush=True)
    Cl_prim = Cl_prim[mask_lmin, ...]
    Cl_lens = Cl_lens[mask_lmin, ...]
    if Sc_approx is not None:
        Sc_approx = Sc_approx[mask_lmin, ...]


    # Start evaluating Fisher                                                                                                                                                                               
    try:
        cov = np.linalg.inv(_mtmm(B_matrix, invNlm, B_matrix))
    # except np.linalg.LinAlgError:
    except:
        cov = np.zeros_like(_mtmm(B_matrix, invNlm, B_matrix))
        good_idx = np.where(np.all(np.diagonal(_mtmm(B_matrix, invNlm, B_matrix), axis1=-1, axis2=-2), axis=-1))
        cov[good_idx] = np.linalg.inv(_mtmm(B_matrix, invNlm[good_idx], B_matrix))

    # print("Test shapes", B_matrix.shape, invNlm.shape, alms.shape, flush=True)
    s = _mv(cov, _mtmv(B_matrix, invNlm, alms)) # True s                                                                                                                                       


    # A_dB = _format_A_dB(B_dB_matrix, x0[:-1]) # mixing matrix derivatives                                                                                                                  

    # Fisher = 0.5*nonparam_biascorr_fisher_bruteforce(B_matrix, s, x0[-1]*Cl_prim+Cl_lens, Sc_approx, A_dB, Cl_prim, comp_of_param, invNlm)

    Fisher = 0.5*nonparam_biascorr_fisher_bruteforce(B_matrix, s, x0[-1]*Cl_prim+Cl_lens, Sc_approx, B_dB_matrix, Cl_prim, invNlm)
    w, v = np.linalg.eig(Fisher)
    print("eigenvalues : ", w)                                                                                                                                                                             
    cond = np.linalg.cond(Fisher)
    print("condition number : ", cond)

    return Fisher


In [38]:
# Fisher_matrix = full_fisher(mixing_matrix_obj, exact_params_mixing_matrix, Cl_lens, Cl_prim, data_map_extended, MICMAC_sampler_obj.lmax, inv_Nl, 
#                             lmin=MICMAC_sampler_obj.lmin, r_start=MICMAC_sampler_obj.r_true, mode='EB', Sc_approx=c_ell_approx)

Fisher_matrix = full_fisher(mixing_matrix_obj, exact_params_mixing_matrix, Cl_lens, Cl_prim, data_map_extended, MICMAC_sampler_obj.lmax, inv_Nl, 
                            lmin=MICMAC_sampler_obj.lmin, r_start=MICMAC_sampler_obj.r_true, mode='B', Sc_approx=c_ell_approx)

1 of 15 complete
2 of 15 complete
3 of 15 complete
4 of 15 complete
5 of 15 complete
6 of 15 complete
7 of 15 complete
8 of 15 complete
9 of 15 complete
10 of 15 complete
11 of 15 complete


12 of 15 complete
13 of 15 complete
14 of 15 complete
15 of 15 complete
Computing mixing matrix
A_dB =  [[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


/var/folders/f8/p6tqlgxj2qn5x_tyjrvc1vnh0000gn/T/ipykernel_81581/2666742401.py:30: RuntimeWarning: invalid value encountered in divide
  m1_dBdB = 2.0*np.trace(_mtmm(A_dB_i, P, _mm(A_dB_j, sst_avg)), axis1=-1, axis2=-2) - Nc_dB*Nc_dB_prime/((Sc+Nc)**2) + Nc_dB*Nc_dB_prime/((Sc_approx+Nc)**2)
/var/folders/f8/p6tqlgxj2qn5x_tyjrvc1vnh0000gn/T/ipykernel_81581/2666742401.py:31: RuntimeWarning: invalid value encountered in divide
  m2_dBdB = 2.0*(Sc-Sc_approx)/((Sc_approx+Nc)*(Sc+Nc))*_mmm(cov, _mmm(_mtmm(A, invN, A_dB_i), cov, _mtmm(A, invN, A_dB_j))
/var/folders/f8/p6tqlgxj2qn5x_tyjrvc1vnh0000gn/T/ipykernel_81581/2666742401.py:35: RuntimeWarning: invalid value encountered in multiply
  m3_dBdB = 2.0/(Sc+Nc)*_mmm(cov, _mmm(_mtmm(A, invN, A_dB_i), sst_avg, _mtmm(A_dB_j, invN, A)), cov)[..., 0, 0]


A_dB =  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
A_dB =  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
A_dB =  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
A_dB =  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
A_dB =  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
A_dB =  [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]

/var/folders/f8/p6tqlgxj2qn5x_tyjrvc1vnh0000gn/T/ipykernel_81581/2666742401.py:43: RuntimeWarning: invalid value encountered in divide
  m_drdr = (Sc_dr/(Sc+Nc))**2


In [39]:
Fisher_matrix

array([[ 9.99062261e+06, -2.41459507e+06, -2.57986236e+06,
        -3.18251678e+06, -2.25616703e+06, -4.18672160e+06,
        -3.51975045e+06, -5.70929348e+05,  8.18719191e+05,
         1.34224670e+06,  4.73037621e+05,  2.54651273e+05,
         2.74846973e+04,  3.02745635e+08, -7.31403912e+07,
        -7.81587924e+07, -9.64431427e+07, -6.84016191e+07,
        -1.27019889e+08, -1.07063516e+08, -1.77107651e+07,
         2.45020757e+07,  4.04161484e+07,  1.42798240e+07,
         7.71367277e+06,  8.65448018e+05,  0.00000000e+00],
       [-2.41459507e+06,  2.54580295e+07, -3.72115939e+06,
        -4.71828348e+06, -3.49306248e+06, -6.91117861e+06,
        -7.16904407e+06, -2.85155773e+06, -3.16361416e+05,
         6.70768677e+05,  3.95266672e+05,  3.11135085e+05,
         1.49535550e+05, -7.31403912e+07,  7.71415875e+08,
        -1.12741101e+08, -1.42977479e+08, -1.05879241e+08,
        -2.09568229e+08, -2.17628120e+08, -8.68040552e+07,
        -9.88329343e+06,  2.00768377e+07,  1.19247610e+

In [40]:
sqrt_inv_Fisher = sp.linalg.sqrtm(np.linalg.pinv(Fisher_matrix))

In [47]:
np.diag(sqrt_inv_Fisher)

array([4.24070113e-04, 2.64151867e-04, 2.04628258e-04, 1.46331922e-04,
       1.33780322e-04, 8.28108741e-05, 6.17099199e-05, 6.33066930e-05,
       7.11086171e-05, 7.48387616e-05, 1.28961204e-04, 1.72586462e-04,
       3.09969946e-04, 6.83180855e-06, 4.25534979e-06, 3.29627672e-06,
       2.35691548e-06, 2.15457139e-06, 1.33313956e-06, 9.92889217e-07,
       1.01856493e-06, 1.14433883e-06, 1.20452164e-06, 2.07686411e-06,
       2.77994538e-06, 4.99354308e-06, 6.41753867e-05])

In [42]:
path_Fisher = '/Users/mag/Documents/PHD1Y/Space_Work/Pixel_non_P2D/MICMAC/test_playground/fisher_litebird_d0s0_lmin2_lmax128_nomask_Alens1.0_r0.0_B_noiselens.txt'
Fisher_matrix_file = np.loadtxt(path_Fisher)


In [45]:
sqrt_inv_Fisher_file = sp.linalg.sqrtm(np.linalg.pinv(Fisher_matrix_file))
np.diag(sqrt_inv_Fisher_file)

array([7.65070504e-04, 5.69208038e-04, 4.51083318e-04, 3.68758218e-04,
       3.17238956e-04, 2.54248989e-04, 2.14573272e-04, 1.75475749e-04,
       1.88980410e-04, 1.74306640e-04, 2.22254998e-04, 3.68792643e-04,
       6.03537133e-04, 1.28560007e-05, 9.56480203e-06, 7.57988281e-06,
       6.19653045e-06, 5.33082835e-06, 4.27237796e-06, 3.60569343e-06,
       2.94872536e-06, 3.17564964e-06, 2.92907771e-06, 3.73476601e-06,
       6.19710900e-06, 1.01416543e-05, 9.22799496e-05])